In [ ]:
# Importing all necessary package
import os
import numpy as np
from PIL import Image

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from cellpose.models import Cellpose
from stardist.models import StarDist2D

from csbdeep.utils import normalize

import tensorflow as tf

from tensorflow import keras
from tensorflow.config import list_physical_devices

# Setting the random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Check if GPU is available
if len(list_physical_devices("GPU")) >= 1:
    use_GPU = True
    tf.config.experimental.set_memory_growth(list_physical_devices("GPU")[0], True)

else:
    use_GPU = False

def load_cellpose():
    """Load the cellpose model and return it"""
    model_c = Cellpose(gpu=use_GPU, model_type="cyto2")
    return model_c


def load_stardist():
    """Load the Stardist model and return it"""
    model_s = StarDist2D.from_pretrained("2D_versatile_he")
    return model_s


def run_cellpose(image, model_cellpose, diameter=None):
    """Run the cellpose model on the image and return the mask"""
    channel = [[0, 0]]
    mask_cellpose, _, _, _ = model_cellpose.eval(
        image, batch_size=1, diameter=diameter, channels=channel
    )
    return mask_cellpose


def run_stardist(image, model_stardist, nms_thresh=0.4, prob_thresh=0.5):
    """Run the stardist model on the image and return the mask"""
    img_norm = image / 255
    img_norm = normalize(img_norm, 1, 99.8)
    mask_stardist, _ = model_stardist.predict_instances(
        img_norm, nms_thresh=nms_thresh, prob_thresh=prob_thresh
    )
    return mask_stardist

model_cellpose = load_cellpose()
model_stardist = load_stardist()

In [ ]:
# Load an image, run CellPose to detect muscle fibers and Stardist to detect nuclei.
try:
    from imageio.v2 import imread
except:
    from imageio import imread

img = imread("sample_img/sample_he.jpg")

# CellPose
mask_cellpose = run_cellpose(
    img, model_cellpose)
mask_cellpose = mask_cellpose.astype(np.uint16)
cellpose_mask_filename = "results/sample_he_cellpose_mask.tiff"
Image.fromarray(mask_cellpose).save(cellpose_mask_filename)

# Stardist
mask_stardist = run_stardist(
    img, model_stardist)
mask_stardist = mask_stardist.astype(np.uint16)
stardist_filename = "results/sample_he_stardist_mask.tiff"
Image.fromarray(mask_stardist).save(stardist_filename)

In [ ]:
# Show  the raw image, the muscle fiber detected and the nuclei detected.
import matplotlib.pyplot as plt
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16,9))
ax1.imshow(img)
ax2.imshow(mask_cellpose)
ax3.imshow(mask_stardist)
plt.show()

In [ ]:
# Based on the Cellpose Image and Stardist image we will extract the coordinates of the nuclei and muscle fibers as a table. 
# We will also extract their area in pixels, perimeter, eccentrictiy....
from skimage.measure import regionprops_table
import pandas as pd
props_cellpose = regionprops_table(
    mask_cellpose,
    properties=[
        "label",
        "area",
        "centroid",
        "eccentricity",
        "bbox",
        "image",
        "perimeter",
    ],
)
df_cellpose = pd.DataFrame(props_cellpose)
df_cellpose

In [ ]:
props_stardist = regionprops_table(
    mask_stardist,
    properties=[
        "label",
        "area",
        "centroid",
        "eccentricity",
        "bbox",
        "image",
        "perimeter",
    ],
)
df_stardist = pd.DataFrame(props_stardist)
df_stardist